labels.npy corrisponde alla struttura che contiene le info di classe, oggetto e posizione sulla griglia. Nel caso del codice che ho su github, 

 0 = row_id, 1 = col_id, 2 = class_id, colonna 3 = object_id

In [2]:
import numpy as np
from sklearn.utils import shuffle
from utils import *

data = np.load('SITS-Missing-Data/D1_balaruc_samples.npy')
masks = np.load('SITS-Missing-Data/D2_balaruc_masks.npy')
lut = np.load('SITS-Missing-Data/D3_balaruc_lut.npy')

In [3]:
data.shape, masks.shape, lut.shape

((137606, 54, 16), (137606, 54), (137606, 4))

In [5]:
def get_unique_ids_by_class(lut):
    labels = lut[:, 1]
    unique_labels = np.unique(labels)
    ids_by_class = {}
    for label in unique_labels:
      idx = np.where(labels == label)
      lut_subset = lut[idx]
      ids_by_class[label] = np.unique(lut_subset[:, 0])
    return ids_by_class

def get_idx_of_object_ids(ids, lut):
    lut_ids = lut[:, 0]
    tot_idx = []
    for i in ids:
        tot_idx.append(np.where(lut_ids == i)[0])
    tot_idx = np.concatenate(tot_idx, axis=0)
    return tot_idx

def get_split_idx(lut, train_perc=.6, val_perc=.2):
  train_idx, valid_idx, test_idx = [], [], []
  unique_ids_by_class = get_unique_ids_by_class(lut)

  for label in unique_ids_by_class:
    ids = unique_ids_by_class[label]
    ids = shuffle(ids)
    
    limit_train = int(len(ids)* train_perc )
    limit_val = limit_train + int(len(ids)* val_perc)
    
    train_idx.extend(get_idx_of_object_ids(ids[0:limit_train], lut))
    valid_idx.extend(get_idx_of_object_ids(ids[limit_train:limit_val], lut))
    test_idx.extend(get_idx_of_object_ids(ids[limit_val::], lut))
  return (train_idx,), (valid_idx,), (test_idx,)

In [29]:
train_idx, val_idx, test_idx = get_split_idx(lut)

In [10]:
data[np.where(masks == 1)] = MISSING_VALUE

num_steps = data.shape[1]
num_bands = data.shape[2]
labels, num_classes = transfer_labels(lut[:, 1])
# labels = convert_to_one_hot(labels, num_classes=len(np.unique(labels)))
prediction_target = data[:, 1:]
mask = np.ones_like(prediction_target)
mask[np.where(prediction_target == MISSING_VALUE)] = 0

In [43]:
for idx in [train_idx, val_idx, test_idx]:
  check_missing_ratio(np.array(mask[idx]))

MISSING RATIO: 0.342
MISSING RATIO: 0.330
MISSING RATIO: 0.338


In [44]:
for idx in [train_idx, val_idx, test_idx]:
  l = labels[idx]
  _, counts = np.unique(l, return_counts=True)
  print([f"{(c/len(l))*100:.1f}%" for c in counts])
  #print([counts[i]/len(l) for i in range(len(counts))])
  #print(np.unique(l, return_counts=True))

['7.3%', '3.7%', '20.5%', '38.8%', '20.9%', '1.5%', '3.5%', '3.7%']
['6.9%', '4.8%', '30.9%', '31.4%', '18.3%', '1.1%', '3.4%', '3.2%']
['8.9%', '3.0%', '25.1%', '29.8%', '23.1%', '1.9%', '5.0%', '3.1%']


In [45]:
for idx in [train_idx, val_idx, test_idx]:
  l = lut[idx]
  print(np.unique(l[:, 0])[0:20])

[ 2  3  5  7  8  9 10 11 13 14 15 16 17 19 21 23 25 26 28 31]
[ 1  4  6 18 22 24 27 29 30 36 43 44 47 49 52 61 64 68 71 73]
[ 0 12 20 34 38 39 42 46 48 54 66 69 74 76 85 95 96 97 98 99]
